In [ ]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import expm_multiply
import os

# ============================================================
# 1. BASIC NUMBER-THEORY FUNCTIONS
# ============================================================

def is_prime(n: int) -> bool:
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

def mobius(n: int) -> int:
    if n == 1:
        return 1
    k = 0
    temp = n
    p = 2
    while p * p <= temp:
        if temp % p == 0:
            temp //= p
            k += 1
            if temp % p == 0:
                return 0
        else:
            p += 1 if p == 2 else 2
    if temp > 1:
        k += 1
    return -1 if k % 2 == 1 else 1

def liouville(n: int) -> int:
    if n <= 1:
        return 1
    count = 0
    temp = n
    p = 2
    while p * p <= temp:
        while temp % p == 0:
            temp //= p
            count += 1
        p += 1 if p == 2 else 2
    if temp > 1:
        count += 1
    return -1 if count % 2 == 1 else 1


# ============================================================
# 2. ULAM GEOMETRY AND POTENTIALS
# ============================================================

def ulam_traversal(size: int):
    y = x = size // 2
    dx = [1, 0, -1, 0]
    dy = [0, -1, 0, 1]
    direction = 0
    step = 1
    taken = 0
    turns = 0
    current = 1
    for _ in range(size * size):
        if 0 <= x < size and 0 <= y < size:
            yield y, x, current
        x += dx[direction]
        y += dy[direction]
        current += 1
        taken += 1
        if taken == step:
            taken = 0
            direction = (direction + 1) % 4
            turns += 1
            if turns % 2 == 0:
                step += 1

def pot_ulam(size, depth):
    grid = np.zeros((size, size), float)
    for y, x, n in ulam_traversal(size):
        if is_prime(n):
            grid[y, x] = depth
    return grid

def pot_mobius(size, depth):
    grid = np.zeros((size, size), float)
    for y, x, n in ulam_traversal(size):
        mu = mobius(n)
        if mu != 0:
            grid[y, x] = depth * mu
    return grid

def pot_liouville(size, depth):
    grid = np.zeros((size, size), float)
    for y, x, n in ulam_traversal(size):
        lam = liouville(n)
        grid[y, x] = depth * lam
    return grid

def pot_cramer(size, depth, seed):
    rng = np.random.default_rng(seed)
    grid = np.zeros((size, size), float)
    for y, x, n in ulam_traversal(size):
        if n == 2:
            grid[y, x] = depth
        elif n > 2:
            p = 1.0 / np.log(n)
            if rng.random() < p:
                grid[y, x] = depth
    return grid


# ============================================================
# 3. HAMILTONIAN AND DYNAMICS
# ============================================================

def build_hamiltonian(potential: np.ndarray, t_hop: float = 1.0):
    N = potential.shape[0]
    size_sq = N * N
    main = 4.0 * t_hop + potential.flatten()
    off1 = -t_hop * np.ones(size_sq - 1, float)
    off1[np.arange(1, size_sq) % N == 0] = 0.0
    offN = -t_hop * np.ones(size_sq - N, float)
    H = diags([main, off1, off1, offN, offN],
              [0, 1, -1, N, -N],
              shape=(size_sq, size_sq),
              format="csc")
    return H

def run_time_series(potential: np.ndarray,
                    times: np.ndarray,
                    sigma: float = 3.0):
    N = potential.shape[0]
    H = build_hamiltonian(potential)
    x, y = np.meshgrid(np.arange(N), np.arange(N))
    cx = cy = N // 2
    psi0 = np.exp(-((x - cx)**2 + (y - cy)**2) / (2 * sigma**2))
    psi0 = psi0.flatten().astype(complex)
    psi0 /= np.linalg.norm(psi0)
    psis = []
    for t in times:
        psi_t = expm_multiply(-1j * H * t, psi0)
        psis.append(psi_t.reshape(N, N))
    return psis


# ============================================================
# 4. OBSERVABLES AND FITTING
# ============================================================

def radial_coords(N: int):
    y, x = np.indices((N, N))
    cx = cy = N // 2
    r2 = (x - cx)**2 + (y - cy)**2
    return r2.astype(float)

def mean_squared_radius(psi: np.ndarray) -> float:
    N = psi.shape[0]
    prob = np.abs(psi)**2
    r2 = radial_coords(N)
    return float((prob * r2).sum())

def participation_ratio(psi: np.ndarray) -> float:
    prob = np.abs(psi.flatten())**2
    return 1.0 / np.sum(prob**2)

def fit_loglog(times: np.ndarray, values: np.ndarray):
    mask = (values > 0) & (times > 0)
    t = times[mask]
    v = values[mask]
    x = np.log(t)
    y = np.log(v)
    A = np.vstack([x, np.ones_like(x)]).T
    alpha, b = np.linalg.lstsq(A, y, rcond=None)[0]
    y_pred = alpha * x + b
    ss_res = np.sum((y - y_pred)**2)
    ss_tot = np.sum((y - y.mean())**2)
    r2 = 1.0 - ss_res / ss_tot if ss_tot > 0 else 0.0
    return alpha, b, r2


# ============================================================
# 5. PARAMETER-SCAN EXPERIMENT
# ============================================================

def run_scan(
    sizes=(81, 121),
    depths=(-1.0, -1.5, -2.0),
    t_max=60.0,
    n_times=15,
    n_runs=8,
    sigma=3.0,
    outdir="scan_results"
):
    """
    Systematic scan over:
      - lattice size
      - potential depth
    For each combination:
      - Ulam, Möbius, Liouville, Cramér
      - n_runs Monte Carlo runs (different seeds for Cramér, depth jitter)
      - compute msr(t), PR(t); fit exponents; save all data.
    """
    os.makedirs(outdir, exist_ok=True)
    landscapes = ["Ulam", "Mobius", "Liouville", "Cramer"]

    times = np.linspace(0, t_max, n_times)[1:]  # skip t=0

    for N in sizes:
        for base_depth in depths:
            tag = f"N{N}_V{base_depth:+.1f}"
            print(f"\n=== SCAN: {tag} ===")
            exponents_msr = {name: [] for name in landscapes}
            exponents_pr = {name: [] for name in landscapes}
            r2_msr = {name: [] for name in landscapes}
            r2_pr = {name: [] for name in landscapes}

            # optionally store some msr/PR curves for later inspection
            sample_curves_msr = {name: [] for name in landscapes}
            sample_curves_pr = {name: [] for name in landscapes}

            for run in range(n_runs):
                print(f"  Run {run+1}/{n_runs}")
                depth_run = base_depth * (1.0 + 0.05 * (run - (n_runs-1)/2))

                pot_u = pot_ulam(N, depth_run)
                pot_m = pot_mobius(N, depth_run)
                pot_l = pot_liouville(N, depth_run)
                pot_c = pot_cramer(N, depth_run, seed=1234 + run)

                all_pots = {
                    "Ulam": pot_u,
                    "Mobius": pot_m,
                    "Liouville": pot_l,
                    "Cramer": pot_c,
                }

                for name, pot in all_pots.items():
                    psis = run_time_series(pot, times, sigma=sigma)
                    msr_vals = np.array([mean_squared_radius(psi) for psi in psis])
                    pr_vals = np.array([participation_ratio(psi) for psi in psis])

                    a_msr, _, r2m = fit_loglog(times, msr_vals)
                    a_pr,  _, r2p = fit_loglog(times, pr_vals)

                    exponents_msr[name].append(a_msr)
                    exponents_pr[name].append(a_pr)
                    r2_msr[name].append(r2m)
                    r2_pr[name].append(r2p)

                    if run == 0:
                        sample_curves_msr[name] = msr_vals
                        sample_curves_pr[name] = pr_vals

            # convert to arrays and save
            savepath = os.path.join(outdir, f"scan_{tag}.npz")
            np.savez_compressed(
                savepath,
                times=times,
                sizes=np.array(sizes),
                depths=np.array(depths),
                exponents_msr={k: np.array(v) for k, v in exponents_msr.items()},
                exponents_pr={k: np.array(v) for k, v in exponents_pr.items()},
                r2_msr={k: np.array(v) for k, v in r2_msr.items()},
                r2_pr={k: np.array(v) for k, v in r2_pr.items()},
                sample_msr=sample_curves_msr,
                sample_pr=sample_curves_pr,
            )
            print(f"  Saved to {savepath}")

            # quick summary to terminal
            print("  Summary exponents:")
            for name in landscapes:
                msr_arr = np.array(exponents_msr[name])
                pr_arr = np.array(exponents_pr[name])
                print(f"    {name:8s} alpha_msr = {msr_arr.mean():.3f} ± {msr_arr.std(ddof=1):.3f}, "
                      f"alpha_PR = {pr_arr.mean():.3f} ± {pr_arr.std(ddof=1):.3f}")


# ============================================================
# 6. MAIN
# ============================================================

if __name__ == "__main__":
    run_scan(
        sizes=(81, 121),         # try more sizes as compute allows
        depths=(-1.0, -1.5, -2.0),
        t_max=60.0,
        n_times=15,
        n_runs=8,               # increase for better statistics
        sigma=3.0,
        outdir="scan_results"
    )


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob

# ============================================================
# 1. LOAD AND PARSE .NPZ FILES (FIXED)
# ============================================================

def load_scan_data(outdir="scan_results"):
    """Load all scan_*.npz files from outdir and return aggregated data."""
    files = sorted(glob(os.path.join(outdir, "scan_*.npz")))
    data = {}
    for filepath in files:
        fname = os.path.basename(filepath)
        print(f"Loading {fname}...")
        loaded = np.load(filepath, allow_pickle=True)

        # Extract tag (e.g., "N81_V-1.0")
        tag = fname.replace("scan_", "").replace(".npz", "")

        # FIX: Use .item() to extract dictionaries from 0-d numpy arrays
        data[tag] = {
            "times": loaded["times"],
            "exponents_msr": loaded["exponents_msr"].item(),
            "exponents_pr": loaded["exponents_pr"].item(),
            "r2_msr": loaded["r2_msr"].item(),
            "r2_pr": loaded["r2_pr"].item(),
            "sample_msr": loaded["sample_msr"].item(),
            "sample_pr": loaded["sample_pr"].item(),
        }
    return data


# ============================================================
# 2. FIGURE 1: REPRESENTATIVE MSR CURVES
# ============================================================

def plot_msr_curves(data, tag="N121_V-1.5", outdir="figures"):
    """Plot mean-squared-radius curves for all landscapes (single representative run)."""
    os.makedirs(outdir, exist_ok=True)

    if tag not in data:
        print(f"Warning: tag {tag} not found in data. Using first available.")
        tag = list(data.keys())[0]

    D = data[tag]
    times = D["times"]

    fig, ax = plt.subplots(figsize=(8, 5))
    colors = {"Ulam": "C0", "Mobius": "C1", "Liouville": "C2", "Cramer": "C3"}

    for landscape, color in colors.items():
        msr_curve = D["sample_msr"][landscape]
        ax.plot(times, msr_curve, marker='o', color=color, label=landscape, linewidth=2)

    ax.set_xlabel("Time", fontsize=12)
    ax.set_ylabel(r"$\langle r^2(t) \rangle$", fontsize=12)
    ax.set_title(f"Mean Squared Radius vs Time ({tag})", fontsize=13)
    ax.legend(fontsize=11)
    ax.grid(alpha=0.3)
    plt.tight_layout()

    savepath = os.path.join(outdir, "fig_msr_curves.pdf")
    plt.savefig(savepath, dpi=300, bbox_inches='tight')
    print(f"Saved: {savepath}")
    plt.close()


# ============================================================
# 3. FIGURE 2: REPRESENTATIVE PR CURVES
# ============================================================

def plot_pr_curves(data, tag="N121_V-1.5", outdir="figures"):
    """Plot participation-ratio curves for all landscapes (single representative run)."""
    os.makedirs(outdir, exist_ok=True)

    if tag not in data:
        tag = list(data.keys())[0]

    D = data[tag]
    times = D["times"]

    fig, ax = plt.subplots(figsize=(8, 5))
    colors = {"Ulam": "C0", "Mobius": "C1", "Liouville": "C2", "Cramer": "C3"}

    for landscape, color in colors.items():
        pr_curve = D["sample_pr"][landscape]
        ax.plot(times, pr_curve, marker='o', color=color, label=landscape, linewidth=2)

    ax.set_xlabel("Time", fontsize=12)
    ax.set_ylabel("Participation Ratio", fontsize=12)
    ax.set_title(f"Participation Ratio vs Time ({tag})", fontsize=13)
    ax.legend(fontsize=11)
    ax.grid(alpha=0.3)
    plt.tight_layout()

    savepath = os.path.join(outdir, "fig_pr_curves.pdf")
    plt.savefig(savepath, dpi=300, bbox_inches='tight')
    print(f"Saved: {savepath}")
    plt.close()


# ============================================================
# 4. FIGURE 3: EXPONENT COMPARISON ACROSS PARAMETERS
# ============================================================

def plot_exponent_heatmaps(data, outdir="figures"):
    """Create heatmaps showing alpha_msr and alpha_PR across parameter space."""
    os.makedirs(outdir, exist_ok=True)

    # Parse tags to extract N and V values
    sizes = set()
    depths = set()
    for tag in data.keys():
        parts = tag.split("_")
        N = int(parts[0][1:])
        V = float(parts[1][1:])
        sizes.add(N)
        depths.add(V)

    sizes = sorted(list(sizes))
    depths = sorted(list(depths))
    landscapes = ["Ulam", "Mobius", "Liouville", "Cramer"]

    # Prepare matrices
    msr_data = {ls: np.zeros((len(sizes), len(depths))) for ls in landscapes}
    pr_data = {ls: np.zeros((len(sizes), len(depths))) for ls in landscapes}

    for i, N in enumerate(sizes):
        for j, V in enumerate(depths):
            tag = f"N{N}_V{V:+.1f}"
            if tag in data:
                for ls in landscapes:
                    msr_arr = data[tag]["exponents_msr"][ls]
                    pr_arr = data[tag]["exponents_pr"][ls]
                    msr_data[ls][i, j] = np.mean(msr_arr)
                    pr_data[ls][i, j] = np.mean(pr_arr)

    # Plot MSR heatmap
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()

    for idx, ls in enumerate(landscapes):
        ax = axes[idx]
        im = ax.imshow(msr_data[ls], cmap='viridis', aspect='auto', origin='lower')
        ax.set_xticks(range(len(depths)))
        ax.set_yticks(range(len(sizes)))
        ax.set_xticklabels([f"{V:.1f}" for V in depths])
        ax.set_yticklabels([f"{N}" for N in sizes])
        ax.set_xlabel("Potential Depth $V_0$", fontsize=10)
        ax.set_ylabel("Lattice Size $N$", fontsize=10)
        ax.set_title(f"{ls}: $\\alpha_{{\\mathrm{{msr}}}}$", fontsize=11)

        # Add colorbar
        cbar = plt.colorbar(im, ax=ax)
        cbar.set_label("Exponent", fontsize=9)

    plt.tight_layout()
    savepath = os.path.join(outdir, "fig_exponents_msr_heatmap.pdf")
    plt.savefig(savepath, dpi=300, bbox_inches='tight')
    print(f"Saved: {savepath}")
    plt.close()

    # Plot PR heatmap
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()

    for idx, ls in enumerate(landscapes):
        ax = axes[idx]
        im = ax.imshow(pr_data[ls], cmap='plasma', aspect='auto', origin='lower')
        ax.set_xticks(range(len(depths)))
        ax.set_yticks(range(len(sizes)))
        ax.set_xticklabels([f"{V:.1f}" for V in depths])
        ax.set_yticklabels([f"{N}" for N in sizes])
        ax.set_xlabel("Potential Depth $V_0$", fontsize=10)
        ax.set_ylabel("Lattice Size $N$", fontsize=10)
        ax.set_title(f"{ls}: $\\alpha_{{\\mathrm{{PR}}}}$", fontsize=11)

        cbar = plt.colorbar(im, ax=ax)
        cbar.set_label("Exponent", fontsize=9)

    plt.tight_layout()
    savepath = os.path.join(outdir, "fig_exponents_pr_heatmap.pdf")
    plt.savefig(savepath, dpi=300, bbox_inches='tight')
    print(f"Saved: {savepath}")
    plt.close()


# ============================================================
# 5. FIGURE 4: EXPONENT BOX PLOTS
# ============================================================

def plot_exponent_boxplots(data, outdir="figures"):
    """Create box plots comparing exponents across landscapes."""
    os.makedirs(outdir, exist_ok=True)

    landscapes = ["Ulam", "Mobius", "Liouville", "Cramer"]

    # Aggregate all exponents
    msr_all = {ls: [] for ls in landscapes}
    pr_all = {ls: [] for ls in landscapes}

    for tag in data.keys():
        for ls in landscapes:
            msr_all[ls].extend(data[tag]["exponents_msr"][ls].tolist())
            pr_all[ls].extend(data[tag]["exponents_pr"][ls].tolist())

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # MSR box plot
    bp1_data = [np.array(msr_all[ls]) for ls in landscapes]
    bp1 = ax1.boxplot(bp1_data, labels=landscapes, patch_artist=True)
    for patch, color in zip(bp1['boxes'], ['C0', 'C1', 'C2', 'C3']):
        patch.set_facecolor(color)
        patch.set_alpha(0.6)
    ax1.set_ylabel(r"$\alpha_{\mathrm{msr}}$", fontsize=12)
    ax1.set_title("Mean Squared Radius Exponent", fontsize=12)
    ax1.grid(axis='y', alpha=0.3)

    # PR box plot
    bp2_data = [np.array(pr_all[ls]) for ls in landscapes]
    bp2 = ax2.boxplot(bp2_data, labels=landscapes, patch_artist=True)
    for patch, color in zip(bp2['boxes'], ['C0', 'C1', 'C2', 'C3']):
        patch.set_facecolor(color)
        patch.set_alpha(0.6)
    ax2.set_ylabel(r"$\alpha_{\mathrm{PR}}$", fontsize=12)
    ax2.set_title("Participation Ratio Exponent", fontsize=12)
    ax2.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    savepath = os.path.join(outdir, "fig_exponent_boxplots.pdf")
    plt.savefig(savepath, dpi=300, bbox_inches='tight')
    print(f"Saved: {savepath}")
    plt.close()


# ============================================================
# 6. SUMMARY STATISTICS TABLE
# ============================================================

def print_summary_table(data):
    """Print a formatted summary table of all exponents."""
    print("\n" + "="*100)
    print("SUMMARY TABLE: EFFECTIVE EXPONENTS ACROSS ALL PARAMETER COMBINATIONS")
    print("="*100)

    landscapes = ["Ulam", "Mobius", "Liouville", "Cramer"]

    for tag in sorted(data.keys()):
        print(f"\n{tag}:")
        print(f"{'Landscape':<12} {'alpha_msr':<20} {'alpha_PR':<20}")
        print("-" * 52)
        for ls in landscapes:
            msr_arr = data[tag]["exponents_msr"][ls]
            pr_arr = data[tag]["exponents_pr"][ls]
            msr_str = f"{np.mean(msr_arr):.3f} ± {np.std(msr_arr, ddof=1):.3f}"
            pr_str = f"{np.mean(pr_arr):.3f} ± {np.std(pr_arr, ddof=1):.3f}"
            print(f"{ls:<12} {msr_str:<20} {pr_str:<20}")


# ============================================================
# 7. MAIN EXECUTION
# ============================================================

if __name__ == "__main__":
    # Load data
    data = load_scan_data(outdir="scan_results")

    print(f"\nLoaded data for {len(data)} parameter configurations:")
    for tag in sorted(data.keys()):
        print(f"  {tag}")

    # Generate all figures
    print("\n\nGenerating figures...")
    plot_msr_curves(data, tag="N121_V-1.5", outdir="figures")
    plot_pr_curves(data, tag="N121_V-1.5", outdir="figures")
    plot_exponent_heatmaps(data, outdir="figures")
    plot_exponent_boxplots(data, outdir="figures")

    # Print summary
    print_summary_table(data)

    print("\n\nAll figures saved to 'figures/' directory.")
    print("You can now view these figures in PDF format.")
